# 📂 Phase 2: Demographic & Housing Data — Data Loading

This notebook loads and merges key **demographic** and **housing-related** variables for California counties from the **U.S. Census API**. The result is an extended county-level GeoDataFrame (`gdf_ca_extended`) that serves as the foundation for further analysis.

🔹 This notebook is **dedicated to data loading only**  
🔹 Data cleaning and exploratory analysis (EDA) will be performed in a **separate notebook**
🔹 **Insurance-related variables** will be collected in a separate notebook (`03_insurance_variables.ipynb`)

---

## 📦 Load Packages


In [3]:
import pandas as pd
import geopandas as gpd
import requests


## 📊 Demographic & Housing Data  
**Source:** U.S. Census Bureau – American Community Survey (ACS 5-Year Estimates, 2023)

The following demographic and housing-related variables were collected via the U.S. Census API. These features enrich the earthquake risk analysis by capturing population dynamics, housing characteristics, and vulnerability indicators.

<details>
<summary><strong>👨‍👩‍👧 Demographic Variables</strong> (click to expand)</summary>

| **Variable Group**       | **Census Table** | **Description**                                                   |
|--------------------------|------------------|-------------------------------------------------------------------|
| Total Population         | B01003           | Total number of people residing in the county                    |
| Median Household Income  | B19013           | Median income in the past 12 months (inflation-adjusted dollars) |
| Age Distribution         | B01001           | Counts of population by age group                                |
| Poverty Level            | B17001           | Number of people living below the federal poverty line           |
| Race & Ethnicity         | B03002           | Population by race and Hispanic origin                           |

</details>

<details>
<summary><strong>🏘️ Housing Variables</strong> (click to expand)</summary>

| **Variable Group**       | **Census Table** | **Description**                                                      |
|--------------------------|------------------|----------------------------------------------------------------------|
| Housing Age              | B25034           | Number of housing units by year built                               |
| Tenure (Owner vs Renter) | B25003           | Counts of owner-occupied vs renter-occupied housing units           |
| Median Home Value        | B25077           | Median value of owner-occupied housing units                        |
| Vacant Housing           | B25004           | Total vacant housing units by vacancy type                          |

</details>

---

## 👨‍👩‍👧 Demographic Variables

### 📊 Total Population (B01003)
**Description:** Total number of residents per county in California.  
**Note:** Serves as a denominator for population density and exposure-based risk modeling.


In [5]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B01003)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_population = pd.DataFrame(data[1:], columns=data[0])

df_population.head()


,B01003_001E,B01003_001EA,B01003_001M,B01003_001MA,GEO_ID,NAME,ucgid
0,1651949,None,-555555555,*****,0500000US06001,"Alameda County, California",0500000US06001
1,1695,None,234,None,0500000US06003,"Alpine County, California",0500000US06003
2,41029,None,-555555555,*****,0500000US06005,"Amador County, California",0500000US06005
3,209470,None,-555555555,*****,0500000US06007,"Butte County, California",0500000US06007
4,45995,None,-555555555,*****,0500000US06009,"Calaveras County, California",0500000US06009


In [6]:
# Rename B01003 columns for clarity
df_population = df_population.rename(columns={
    "B01003_001E": "total_population",  # Total population per county
    "NAME": "county"
})

# Convert total population to numeric
df_population["total_population"] = pd.to_numeric(df_population["total_population"], errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_population["COUNTYFP"] = df_population["GEO_ID"].str[-3:].astype(str)

# Load California counties GeoJSON created in Phase 1 (01_data_collection)
gdf_ca_extended = gpd.read_file("../output/california_counties.geojson")

gdf_ca_extended.head()


,STATEFP,COUNTYFP,COUNTYNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,091,00277310,06091,0500000US06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694578,23299110,+39.5769252,-120.5219926,"POLYGON ((-120.55587 39.50874, -120.55614 39.5..."
1,06,067,00277298,06067,0500000US06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2500063005,75323439,+38.4501363,-121.3443291,"POLYGON ((-121.43991 38.25553, -121.44002 38.2..."
2,06,083,00277306,06083,0500000US06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7080874935,2729198796,+34.5366774,-120.0383645,"MULTIPOLYGON (((-120.58226 34.10752, -120.5790..."
3,06,009,01675885,06009,0500000US06009,Calaveras,Calaveras County,06,H1,G4020,None,None,None,A,2641829627,43797225,+38.1910682,-120.5541065,"POLYGON ((-120.6318 38.34603, -120.63066 38.34..."
4,06,111,00277320,06111,0500000US06111,Ventura,Ventura County,06,H1,G4020,348,37100,None,A,4767585000,947376202,+34.3587477,-119.1331453,"MULTIPOLYGON (((-119.63607 33.28071, -119.6348..."


In [7]:
# Rename county name column for consistency
gdf_ca_extended = gdf_ca_extended.rename(columns={"NAME": "county"})

# Ensure COUNTYFP is string type for accurate merging
gdf_ca_extended["COUNTYFP"] = gdf_ca_extended["COUNTYFP"].astype(str)

# Merge total population into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_population[["COUNTYFP", "total_population"]],
    how="left",
    on="COUNTYFP"
)


✅ `total_population` successfully merged into `gdf_ca_extended`.  
Use `gdf_ca_extended[['county', 'total_population']]` to view or validate.


In [9]:
# ✅ Quick preview of top 10 counties by population for validation
gdf_ca_extended[["county", "total_population"]].sort_values("total_population", ascending=False).head(10)


,county,total_population
5,Los Angeles,9848406
8,San Diego,3282782
47,Orange,3164063
31,Riverside,2449909
23,San Bernardino,2187816
38,Santa Clara,1903297
41,Alameda,1651949
1,Sacramento,1584047
24,Contra Costa,1161458
36,Fresno,1012152


---

### 💰 Median Household Income (B19013)
**Description:** Median household income in the past 12 months, adjusted to 2023 dollars.  
**Note:** Proxy for socioeconomic status and insurance affordability.


In [12]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B19013)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_income = pd.DataFrame(data[1:], columns=data[0])

df_income.head()


,B19013_001E,B19013_001EA,B19013_001M,B19013_001MA,GEO_ID,NAME,ucgid
0,126240,None,1364,None,0500000US06001,"Alameda County, California",0500000US06001
1,110781,None,18601,None,0500000US06003,"Alpine County, California",0500000US06003
2,81526,None,7398,None,0500000US06005,"Amador County, California",0500000US06005
3,68574,None,2351,None,0500000US06007,"Butte County, California",0500000US06007
4,79877,None,4638,None,0500000US06009,"Calaveras County, California",0500000US06009


In [13]:
# Rename B19013 columns for clarity
df_income = df_income.rename(columns={
    "B19013_001E": "median_income",  # Median household income (in inflation-adjusted dollars)
    "NAME": "county",
})

# Convert median income to numeric
df_income["median_income"] = pd.to_numeric(df_income["median_income"], errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_income["COUNTYFP"] = df_income["GEO_ID"].str[-3:].astype(str)

# # Merge median income into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_income[["COUNTYFP", "median_income"]],
    how="left",
    on="COUNTYFP"
)


✅ `median_income` successfully merged into `gdf_ca_extended`.  
Use `gdf_ca_extended[['county', 'median_income']]` to view or validate.


In [15]:
# ✅ Quick preview of top 10 counties by median household income
gdf_ca_extended[["county", "median_income"]].sort_values("median_income", ascending=False).head(10)


,county,median_income
38,Santa Clara,159674
53,San Mateo,156000
11,Marin,142785
10,San Francisco,141446
41,Alameda,126240
24,Contra Costa,125727
9,Placer,114678
47,Orange,113702
25,Alpine,110781
55,Santa Cruz,109266



---


### 📈 Age Distribution (B01001)
**Description:** Number of residents grouped by age (0–17, 18–64, 65+).  
**Note:** Helps identify vulnerable populations, especially children and elderly.


In [18]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B01001)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_age = pd.DataFrame(data[1:], columns=data[0])

df_age.head()


,B01001_001E,B01001_001EA,B01001_001M,B01001_001MA,B01001_002E,B01001_002EA,B01001_002M,B01001_002MA,B01001_003E,B01001_003EA,...,B01001_048EA,B01001_048M,B01001_048MA,B01001_049E,B01001_049EA,B01001_049M,B01001_049MA,GEO_ID,NAME,ucgid
0,1651949,None,-555555555,*****,819520,None,-555555555,*****,45077,None,...,None,764,None,19054,None,924,None,0500000US06001,"Alameda County, California",0500000US06001
1,1695,None,234,None,885,None,148,None,73,None,...,None,5,None,9,None,13,None,0500000US06003,"Alpine County, California",0500000US06003
2,41029,None,-555555555,*****,22409,None,146,None,849,None,...,None,157,None,718,None,198,None,0500000US06005,"Amador County, California",0500000US06005
3,209470,None,-555555555,*****,104228,None,86,None,5641,None,...,None,344,None,2264,None,237,None,0500000US06007,"Butte County, California",0500000US06007
4,45995,None,-555555555,*****,22864,None,298,None,999,None,...,None,225,None,735,None,228,None,0500000US06009,"Calaveras County, California",0500000US06009


In [19]:
# Rename B01001 columns for clarity
df_age = df_age.rename(columns={
    "B01001_001E": "total_population",  # Total population (age universe)
    "NAME": "county"                               
})

# Identify all age-related columns in B01001 table
age_columns = [col for col in df_age.columns if col.startswith("B01001_") and col.endswith("E")]

# Convert age columns to numeric
df_age[age_columns] = df_age[age_columns].apply(pd.to_numeric, errors="coerce")
df_age["total_population"] = pd.to_numeric(df_age["total_population"], errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_age["COUNTYFP"] = df_age["GEO_ID"].str[-3:].astype(str)

# Define age groupings by ACS variable codes
age_0_17_cols = [f"B01001_{str(i).zfill(3)}E" for i in range(3, 7)] + \
                [f"B01001_{str(i).zfill(3)}E" for i in range(27, 31)]

age_18_64_cols = [f"B01001_{str(i).zfill(3)}E" for i in range(7, 20)] + \
                 [f"B01001_{str(i).zfill(3)}E" for i in range(31, 44)]

age_65_up_cols = [f"B01001_{str(i).zfill(3)}E" for i in range(20, 26)] + \
                 [f"B01001_{str(i).zfill(3)}E" for i in range(44, 50)]

# Aggregate age groups (total counts)
age_0_17 = df_age[age_0_17_cols].astype(float).sum(axis=1).rename("age_0_17")
age_18_64 = df_age[age_18_64_cols].astype(float).sum(axis=1).rename("age_18_64")
age_65_up = df_age[age_65_up_cols].astype(float).sum(axis=1).rename("age_65_up")

# Add all new columns at once (avoids fragmentation warnings)
df_age = pd.concat([df_age, age_0_17, age_18_64, age_65_up], axis=1)

# Calculate age group percentages
df_age["age_0_17_pct"] = df_age["age_0_17"] / df_age["total_population"] * 100
df_age["age_18_64_pct"] = df_age["age_18_64"] / df_age["total_population"] * 100
df_age["age_65_up_pct"] = df_age["age_65_up"] / df_age["total_population"] * 100

# Merge age totals and percentages into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_age[[
        "COUNTYFP",
        "age_0_17", "age_18_64", "age_65_up",
        "age_0_17_pct", "age_18_64_pct", "age_65_up_pct"
    ]],
    on="COUNTYFP",
    how="left"
)


✅ Age group variables successfully merged into `gdf_ca_extended`.  
Includes raw counts and percentages.  
Use:  
`gdf_ca_extended[['county', 'age_0_17', 'age_18_64', 'age_65_up']]`  
`gdf_ca_extended[['county', 'age_0_17_pct', 'age_18_64_pct', 'age_65_up_pct']]`
to view or validate


In [21]:
# ✅ Preview age distribution totals and percentages for validation
gdf_ca_extended[['county', 'age_0_17', 'age_18_64', 'age_65_up', 'age_0_17_pct', 'age_18_64_pct', 'age_65_up_pct']].head()


,county,age_0_17,age_18_64,age_65_up,age_0_17_pct,age_18_64_pct,age_65_up_pct
0,Sierra,349.0,1440.0,942.0,12.779202,52.727938,34.492860
1,Sacramento,365667.0,981913.0,236467.0,23.084353,61.987618,14.928029
2,Santa Barbara,99608.0,272385.0,71982.0,22.435497,61.351427,16.213075
3,Calaveras,7768.0,25007.0,13220.0,16.888792,54.368953,28.742255
4,Ventura,185795.0,511149.0,141315.0,22.164391,60.977454,16.858155



---


### 📉 Poverty Level (B17001)
**Description:** Total number of individuals living below the federal poverty line.  
**Note:** Indicates economic vulnerability and potential underinsurance.


In [24]:
# Request data from API
url = 'https://api.census.gov/data/2023/acs/acs5?get=group(B17001)&ucgid=pseudo(0400000US06$0500000)'
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_poverty = pd.DataFrame(data[1:], columns=data[0])

df_poverty.head()


,B17001_001E,B17001_001EA,B17001_001M,B17001_001MA,B17001_002E,B17001_002EA,B17001_002M,B17001_002MA,B17001_003E,B17001_003EA,...,B17001_058EA,B17001_058M,B17001_058MA,B17001_059E,B17001_059EA,B17001_059M,B17001_059MA,GEO_ID,NAME,ucgid
0,1624096,None,962,None,149752,None,4801,None,68117,None,...,None,680,None,48662,None,759,None,0500000US06001,"Alameda County, California",0500000US06001
1,1695,None,234,None,209,None,98,None,147,None,...,None,42,None,15,None,15,None,0500000US06003,"Alpine County, California",0500000US06003
2,37799,None,79,None,2931,None,601,None,1354,None,...,None,157,None,2117,None,125,None,0500000US06005,"Amador County, California",0500000US06005
3,204601,None,258,None,37531,None,1779,None,17187,None,...,None,325,None,7074,None,242,None,0500000US06007,"Butte County, California",0500000US06007
4,45528,None,140,None,6083,None,1035,None,2911,None,...,None,176,None,2157,None,168,None,0500000US06009,"Calaveras County, California",0500000US06009


In [25]:
# Rename B17001 columns for clarity
df_poverty = df_poverty.rename(columns={
    "B17001_001E": "total_population",      # Population for whom poverty status is determined
    "B17001_002E": "below_poverty",         # Population below the poverty line
    "NAME": "county"
})

# Convert population counts to numeric
df_poverty[["total_population", "below_poverty"]] = df_poverty[["total_population", "below_poverty"]].apply(pd.to_numeric, errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_poverty["COUNTYFP"] = df_poverty["GEO_ID"].str[-3:].astype(str)

# Calculate percent of population below poverty line
df_poverty["below_poverty_pct"] = (df_poverty["below_poverty"] / df_poverty["total_population"]) * 100

# Merge poverty rate and count into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_poverty[["COUNTYFP", "below_poverty", "below_poverty_pct"]],
    how="left",
    on="COUNTYFP"
)


✅ Poverty variables successfully merged into `gdf_ca_extended`.  
Includes raw and percent values.  
Use: `gdf_ca_extended[['county', 'below_poverty', 'below_poverty_pct']]` to view or validate.


In [27]:
# ✅ Preview poverty totals and percentage — sorted by highest poverty rate
gdf_ca_extended[['county', 'below_poverty', 'below_poverty_pct']].sort_values("below_poverty_pct", ascending=False).head()


,county,below_poverty,below_poverty_pct
35,Modoc,1717,20.276334
37,Madera,30187,19.864050
48,Imperial,33705,19.641723
32,Kern,168825,19.047538
30,Humboldt,25064,18.907094


---


### 🌎 Race & Ethnicity (B03002)
**Description:** Population counts by race and Hispanic/Latino ethnicity.  
**Note:** Useful for evaluating social vulnerability and equity in coverage.


In [30]:
# Request data from API
url = 'https://api.census.gov/data/2023/acs/acs5?get=group(B03002)&ucgid=pseudo(0400000US06$0500000)'
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_race = pd.DataFrame(data[1:], columns=data[0])

df_race.head()


,B03002_001E,B03002_001EA,B03002_001M,B03002_001MA,B03002_002E,B03002_002EA,B03002_002M,B03002_002MA,B03002_003E,B03002_003EA,...,B03002_020EA,B03002_020M,B03002_020MA,B03002_021E,B03002_021EA,B03002_021M,B03002_021MA,GEO_ID,NAME,ucgid
0,1651949,None,-555555555,*****,1266704,None,-555555555,*****,466445,None,...,None,4202,None,20890,None,1802,None,0500000US06001,"Alameda County, California",0500000US06001
1,1695,None,234,None,1446,None,218,None,993,None,...,None,76,None,15,None,15,None,0500000US06003,"Alpine County, California",0500000US06003
2,41029,None,-555555555,*****,34668,None,-555555555,*****,30234,None,...,None,515,None,790,None,284,None,0500000US06005,"Amador County, California",0500000US06005
3,209470,None,-555555555,*****,168641,None,-555555555,*****,139527,None,...,None,1135,None,2838,None,668,None,0500000US06007,"Butte County, California",0500000US06007
4,45995,None,-555555555,*****,39592,None,-555555555,*****,35599,None,...,None,606,None,250,None,176,None,0500000US06009,"Calaveras County, California",0500000US06009


In [31]:
# Rename B03002 columns for clarity
df_race = df_race.rename(columns={
    "B03002_001E": "total_population_race",      # Total population (race/ethnicity universe)
    "B03002_003E": "white_alone",                # White alone, not Hispanic or Latino
    "B03002_004E": "black_alone",                # Black or African American alone
    "B03002_005E": "american_indian_alone",      # American Indian and Alaska Native alone
    "B03002_006E": "asian_alone",                # Asian alone
    "B03002_007E": "native_hawaiian_alone",      # Native Hawaiian and Pacific Islander alone
    "B03002_012E": "hispanic",                   # Hispanic or Latino (of any race)
    "NAME": "county"
})

# Select race/ethnicity columns to convert
race_cols = [
    "total_population_race", "white_alone", "black_alone",
    "american_indian_alone", "asian_alone",
    "native_hawaiian_alone", "hispanic"
]

# Convert selected columns to numeric
df_race[race_cols] = df_race[race_cols].apply(pd.to_numeric, errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_race["COUNTYFP"] = df_race["GEO_ID"].str[-3:].astype(str)

# Calculate share of each race/ethnicity group as a percent of total population
for col in race_cols[1:]:  
    df_race[f"{col}_pct"] = (df_race[col] / df_race["total_population_race"]) * 100

# Select columns to merge into main GeoDataFrame
cols_to_merge = ["COUNTYFP"] + [col for col in df_race.columns if col.endswith("_pct")]

# Merge percentage columns into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_race[cols_to_merge], 
    on="COUNTYFP", 
    how="left"
)


✅ Race & ethnicity variables successfully merged into `gdf_ca_extended`.  
Includes percent population by group.  
Use:  
`gdf_ca_extended.filter(like='_pct')` or  
`gdf_ca_extended[['county', 'white_alone_pct', 'black_alone_pct', 'asian_alone_pct', 'hispanic_pct']]`
to preview or validate.
 

In [33]:
# ✅ Preview all percentage-based demographic variables
gdf_ca_extended[["county"] + list(gdf_ca_extended.filter(like='_pct').columns)].head()


,county,age_0_17_pct,age_18_64_pct,age_65_up_pct,below_poverty_pct,white_alone_pct,black_alone_pct,american_indian_alone_pct,asian_alone_pct,native_hawaiian_alone_pct,hispanic_pct
0,Sierra,12.779202,52.727938,34.492860,12.343335,88.831930,0.292933,0.000000,0.000000,0.000000,9.593555
1,Sacramento,23.084353,61.987618,14.928029,12.615335,41.530649,9.126181,0.264260,17.208012,1.091255,23.972395
2,Santa Barbara,22.435497,61.351427,16.213075,13.772858,41.519455,1.538825,0.229743,5.145785,0.086041,47.617997
3,Calaveras,16.888792,54.368953,28.742255,13.361009,77.397543,1.150125,0.921839,2.317643,0.052180,13.921078
4,Ventura,22.164391,60.977454,16.858155,9.017859,42.867658,1.674065,0.157946,7.066312,0.150073,43.842774


---


## 🏘️ Housing Variables

### 🏚️ Housing Age (B25034)
**Description:** Housing units by year built, grouped by construction period.  
**Note:** Older homes may be more structurally vulnerable.


In [36]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B25034)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_housing_age = pd.DataFrame(data[1:], columns=data[0])

df_housing_age.head()


,B25034_001E,B25034_001EA,B25034_001M,B25034_001MA,B25034_002E,B25034_002EA,B25034_002M,B25034_002MA,B25034_003E,B25034_003EA,...,B25034_010EA,B25034_010M,B25034_010MA,B25034_011E,B25034_011EA,B25034_011M,B25034_011MA,GEO_ID,NAME,ucgid
0,630726,None,295,None,5484,None,504,None,39427,None,...,None,1567,None,121185,None,2087,None,0500000US06001,"Alameda County, California",0500000US06001
1,1587,None,96,None,11,None,17,None,34,None,...,None,31,None,89,None,43,None,0500000US06003,"Alpine County, California",0500000US06003
2,18919,None,47,None,136,None,124,None,846,None,...,None,160,None,1459,None,287,None,0500000US06005,"Amador County, California",0500000US06005
3,91176,None,88,None,1568,None,333,None,6687,None,...,None,538,None,6124,None,531,None,0500000US06007,"Butte County, California",0500000US06007
4,27579,None,32,None,146,None,117,None,1482,None,...,None,188,None,939,None,222,None,0500000US06009,"Calaveras County, California",0500000US06009


In [37]:
# Define B25034 columns for housing unit counts by year built
housing_cols = [f"B25034_{str(i).zfill(3)}E" for i in range(2, 12)]  # B25034_002E to B25034_011E

# Convert year-built columns to numeric
df_housing_age[housing_cols] = df_housing_age[housing_cols].apply(pd.to_numeric, errors="coerce")

# Aggregate housing units built before 1980
df_housing_age["built_pre_1980"] = df_housing_age[[
    'B25034_007E',    # 1970–1979 
    'B25034_008E',    # 1960–1969
    'B25034_009E',    # 1950–1959
    'B25034_010E',    # 1940–1949
    'B25034_011E'     # 1939 or earlier
]].sum(axis=1)

# Aggregate housing units built between 1980 and 1999
df_housing_age["built_1980_1999"] = df_housing_age[[
    'B25034_005E',    # 1990–1999 
    'B25034_006E'     # 1980–1989
]].sum(axis=1)

# Aggregate housing units built in 2000 or later
df_housing_age["built_2000_up"] = df_housing_age[[
    'B25034_002E',    # 2020 or later 
    'B25034_003E',    # 2010–2019 
    'B25034_004E'     # 2000–2009
]].sum(axis=1)

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_housing_age["COUNTYFP"] = df_housing_age["GEO_ID"].str[-3:].astype(str)

# Merge aggregated housing age groups into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_housing_age[["COUNTYFP", "built_pre_1980", "built_1980_1999", "built_2000_up"]],
    how="left",
    on="COUNTYFP"
)


✅ Housing age groups successfully merged into `gdf_ca_extended`.  
Use `gdf_ca_extended[['county', 'built_pre_1980', 'built_1980_1999', 'built_2000_up']]` to view or validate.


In [39]:
# ✅ Preview housing age group totals by county
gdf_ca_extended[['county', 'built_pre_1980', 'built_1980_1999', 'built_2000_up']].head()


,county,built_pre_1980,built_1980_1999,built_2000_up
0,Sierra,1202,554,343
1,Sacramento,292147,174296,127173
2,Santa Barbara,97171,39480,23115
3,Calaveras,9227,11634,6718
4,Ventura,165876,82384,46391


---


### 🏠 Tenure – Owner vs Renter (B25003)
**Description:** Counts of owner-occupied vs renter-occupied housing units.  
**Note:** Renters may have lower insurance coverage or less structural control.


In [42]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B25003)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_tenure = pd.DataFrame(data[1:], columns=data[0])

df_tenure.head()


,B25003_001E,B25003_001EA,B25003_001M,B25003_001MA,B25003_002E,B25003_002EA,B25003_002M,B25003_002MA,B25003_003E,B25003_003EA,B25003_003M,B25003_003MA,GEO_ID,NAME,ucgid
0,593117,None,1676,None,320712,None,2874,None,272405,None,2581,None,0500000US06001,"Alameda County, California",0500000US06001
1,473,None,103,None,383,None,92,None,90,None,52,None,0500000US06003,"Alpine County, California",0500000US06003
2,16066,None,341,None,12863,None,457,None,3203,None,472,None,0500000US06005,"Amador County, California",0500000US06005
3,82345,None,797,None,47987,None,1099,None,34358,None,1058,None,0500000US06007,"Butte County, California",0500000US06007
4,17897,None,567,None,15099,None,612,None,2798,None,394,None,0500000US06009,"Calaveras County, California",0500000US06009


In [43]:
# Rename columns for clarity
df_tenure = df_tenure.rename(columns={
    "B25003_001E": "total_housing_units",     # Total occupied housing units
    "B25003_002E": "owner_occupied",          # Owner-occupied units
    "B25003_003E": "renter_occupied",         # Renter-occupied units
    "NAME": "county"   
})

# Convert housing unit counts to numeric
cols_to_num = ["total_housing_units", "owner_occupied", "renter_occupied"]
df_tenure[cols_to_num] = df_tenure[cols_to_num].apply(pd.to_numeric, errors="coerce")

# Extract 3-digit county FIPS and ensure string format
df_tenure["COUNTYFP"] = df_tenure["GEO_ID"].str[-3:].astype(str)

# Calculate owner and renter occupancy as a percentage of total occupied units
df_tenure["owner_pct"] = (df_tenure["owner_occupied"] / df_tenure["total_housing_units"]) * 100
df_tenure["renter_pct"] = (df_tenure["renter_occupied"] / df_tenure["total_housing_units"]) * 100

# Merge housing tenure variables into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_tenure[[
        "COUNTYFP", "total_housing_units", "owner_occupied", "renter_occupied", 
        "owner_pct", "renter_pct"
    ]],
    how="left",
    on="COUNTYFP"
)


✅ Housing tenure variables successfully merged into `gdf_ca_extended`.  
Includes counts and occupancy percentages.  
Use `gdf_ca_extended[['county', 'owner_pct', 'renter_pct']]` to view or validate.


In [45]:
# ✅ Preview owner vs renter housing statistics
gdf_ca_extended[['county', 'total_housing_units', 'owner_pct', 'renter_pct']].head()


,county,total_housing_units,owner_pct,renter_pct
0,Sierra,1206,78.275290,21.724710
1,Sacramento,568223,58.628919,41.371081
2,Santa Barbara,148960,52.869227,47.130773
3,Calaveras,17897,84.366095,15.633905
4,Ventura,278045,64.157960,35.842040


---


### 🏡 Median Home Value (B25077)
**Description:** Median value of owner-occupied housing units.  
**Note:** Proxy for insured property value and affluence.


In [48]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B25077)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_home_value = pd.DataFrame(data[1:], columns=data[0])

df_home_value.head()


,B25077_001E,B25077_001EA,B25077_001M,B25077_001MA,GEO_ID,NAME,ucgid
0,1057400,None,6624,None,0500000US06001,"Alameda County, California",0500000US06001
1,466100,None,52099,None,0500000US06003,"Alpine County, California",0500000US06003
2,422800,None,15875,None,0500000US06005,"Amador County, California",0500000US06005
3,408700,None,8846,None,0500000US06007,"Butte County, California",0500000US06007
4,441800,None,13276,None,0500000US06009,"Calaveras County, California",0500000US06009


In [49]:
# Rename B25077 column for clarity
df_home_value = df_home_value.rename(columns={
    "B25077_001E": "median_home_value",  # Median value of owner-occupied housing units
    "NAME": "county"
})

# Convert median home value to numeric
df_home_value["median_home_value"] = pd.to_numeric(df_home_value["median_home_value"], errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_home_value["COUNTYFP"] = df_home_value["GEO_ID"].str[-3:].astype(str)

# Merge median home value into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_home_value[["COUNTYFP", "median_home_value"]], 
    on="COUNTYFP", 
    how="left"
)


✅ `median_home_value` successfully merged into `gdf_ca_extended`.  
Use `gdf_ca_extended[['county', 'median_home_value']]` to view or validate.


In [51]:
# ✅ Quick preview of top 10 counties by median home value
gdf_ca_extended[["county", "median_home_value"]].sort_values("median_home_value", ascending=False).head(10)


,county,median_home_value
53,San Mateo,1494500
11,Marin,1390000
38,Santa Clara,1382800
10,San Francisco,1380500
41,Alameda,1057400
55,Santa Cruz,1015200
47,Orange,915500
14,Napa,838800
24,Contra Costa,830800
8,San Diego,791600


---


### 🚪 Vacant Housing Units (B25004)
**Description:** Total vacant units, categorized by type (e.g., for rent, for sale, seasonal).  
**Note:** High vacancy may suggest instability or seasonal exposure.


In [54]:
# Request data from API
url = "https://api.census.gov/data/2023/acs/acs5?get=group(B25004)&ucgid=pseudo(0400000US06$0500000)"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_vacant = pd.DataFrame(data[1:], columns=data[0])

df_vacant.head()

,B25004_001E,B25004_001EA,B25004_001M,B25004_001MA,B25004_002E,B25004_002EA,B25004_002M,B25004_002MA,B25004_003E,B25004_003EA,...,B25004_007EA,B25004_007M,B25004_007MA,B25004_008E,B25004_008EA,B25004_008M,B25004_008MA,GEO_ID,NAME,ucgid
0,37609,None,1623,None,15116,None,1242,None,1953,None,...,None,32,None,12632,None,1062,None,0500000US06001,"Alameda County, California",0500000US06001
1,1114,None,100,None,10,None,13,None,9,None,...,None,14,None,27,None,23,None,0500000US06003,"Alpine County, California",0500000US06003
2,2853,None,343,None,110,None,97,None,60,None,...,None,29,None,433,None,171,None,0500000US06005,"Amador County, California",0500000US06005
3,8831,None,792,None,2147,None,474,None,793,None,...,None,22,None,2720,None,448,None,0500000US06007,"Butte County, California",0500000US06007
4,9682,None,568,None,171,None,112,None,43,None,...,None,32,None,951,None,261,None,0500000US06009,"Calaveras County, California",0500000US06009


In [55]:
# Rename B25004 columns for clarity
df_vacant = df_vacant.rename(columns={
    "B25004_001E": "vacant_total",            # Total vacant housing units
    "B25004_002E": "for_rent",                # Vacant for rent
    "B25004_003E": "rented_not_occupied",     # Rented but not yet occupied
    "B25004_004E": "for_sale",                # Vacant for sale
    "B25004_005E": "sold_not_occupied",       # Sold but not yet occupied
    "B25004_006E": "seasonal",                # For seasonal, recreational, or occasional use
    "B25004_007E": "migrant",                 # For migrant workers
    "B25004_008E": "other_vacant",            # Other vacant units
    "NAME": "county"
})

# Convert all vacancy type columns to numeric
cols_to_numeric = [
    "vacant_total", "for_rent", "rented_not_occupied", "for_sale",
    "sold_not_occupied", "seasonal", "migrant", "other_vacant"
]
df_vacant[cols_to_numeric] = df_vacant[cols_to_numeric].apply(pd.to_numeric, errors="coerce")

# Extract 3-digit county FIPS from GEO_ID and ensure string format
df_vacant["COUNTYFP"] = df_vacant["GEO_ID"].str[-3:].astype(str)

# Calculate percentage of each vacancy type relative to total vacant units
vacant_cols = [
    "for_rent", "rented_not_occupied", "for_sale",
    "sold_not_occupied", "seasonal", "migrant", "other_vacant"
]
for col in vacant_cols:
    df_vacant[f"{col}_pct"] = (df_vacant[col] / df_vacant["vacant_total"]) * 100

# Merge vacancy percentages into main GeoDataFrame
gdf_ca_extended = gdf_ca_extended.merge(
    df_vacant[["COUNTYFP"] + [f"{col}_pct" for col in vacant_cols]],
    on="COUNTYFP",
    how="left"
)


✅ Vacancy type percentages successfully merged into `gdf_ca_extended`.  
Use:  
`gdf_ca_extended.filter(like='_pct')` or  
`gdf_ca_extended[['county', 'for_rent_pct', 'for_sale_pct', 'seasonal_pct']]`
to view or validate.


In [57]:
# ✅ Preview selected vacant housing percentage types
gdf_ca_extended[['county', "for_rent_pct", "for_sale_pct"]].head()


,county,for_rent_pct,for_sale_pct
0,Sierra,0.000000,4.143337
1,Sacramento,34.907258,8.187296
2,Santa Barbara,20.636683,5.996669
3,Calaveras,1.766164,2.891964
4,Ventura,23.750452,5.654583


---


In [59]:
# Save extended GeoDataFrame
gdf_ca_extended.to_file("../output/gdf_ca_extended.geojson", driver="GeoJSON")
gdf_ca_extended.to_csv("../output/gdf_ca_extended.csv", index=False)

print("✅ gdf_ca_extended saved as both GeoJSON and CSV in /output/")


✅ gdf_ca_extended saved as both GeoJSON and CSV in /output/
